In [1]:
import numpy as np
import os, sys, subprocess
import pandas as pd
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from tqdm import tqdm

import config

from pathlib import Path
home = str(Path.home())
original_path = os.getcwd()
MainPath = home+'/sed_fitting/'

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
class FitsUtils:
    def __init__(self, fits_path, filtername, kind = 'sci'):
        import numpy as np
        from astropy.io import fits
        from astropy.wcs import WCS
        self.fits_path = fits_path
        self.filtername = filtername
        self.kind = kind
        self.fitsfile = fits.open(self.fits_path)
        if kind == 'sci':
            self.signal_with_nans = self.fitsfile[0].data
            self.signal = np.nan_to_num(self.signal_with_nans)
            self.hdr = self.fitsfile[0].header
        elif kind == 'err': # Err map does not have a valid hdr, so I appizzo the one from 'sci'
            self.signal_with_nans = self.fitsfile[0].data
            self.signal = np.nan_to_num(self.signal_with_nans)
            scipath = fits_path[:-12]+'_sci.fits.gz'
            self.hdr = fits.getheader(scipath)
        elif kind == 'wht':
            self.signal_with_nans = np.sqrt(np.absolute(1/fits.getdata(self.fits_path)))
            self.signal = np.nan_to_num(self.signal_with_nans)
            scipath = signal_path[:-12]+'_sci.fits.gz'
            self.hdr = fits.open(scipath)[0].header
        self.wcs = WCS(self.hdr)
        self.telescope = self.hdr['TELESCOP']
        return

In [3]:
working_filters = ['f435w', 'f475w', 'f606w', 'f814w', 'f090w', \
                   'f115w', 'f150w', 'f200w', 'f277w', 'f356w', \
                   'f410m', 'f444w', 'f770w', 'f1800w']

bundle = [(config.get_map_path(filter, 'sci'), filter, 'sci') for filter in working_filters]
df_bundle = pd.DataFrame(bundle, columns = ['Path', 'Filter', 'kind'])

Generate and save cutouts.

In [4]:
def generate_cutouts(fits_file, cutout_size, df):
    print('Cutouts for filter {0} {1} {2}'.format(fits_file.telescope, fits_file.filtername, fits_file.kind))
    file_name = '{0}_{1}_{2}'.format(fits_file.telescope, fits_file.filtername, fits_file.kind)
    coords = SkyCoord(df.RAJ2000.values*u.deg, df.DEJ2000.values*u.deg, frame = 'icrs')
    for id_source, coord in zip(df.source_id.values, coords):
        if os.path.exists('Galaxies/{0}/{1}.fits'.format(id_source, file_name)): continue
        try: cutout = Cutout2D(fits_file.signal, coord, cutout_size, fits_file.wcs, fill_value='nan')
        except: continue
        if cutout.data.ptp() == 0 or \
            cutout.data[int(cutout.center_cutout[0]), int(cutout.center_cutout[1])] == 0:
            print('No source in this cutout')
            continue
        hdu = fits.ImageHDU()
        hdu.data = cutout.data
        hdu.header = cutout.wcs.to_header()
        hdu.header['CD1_1'] = fits_file.hdr['CD1_1']
        hdu.header['CD2_2'] = fits_file.hdr['CD2_2']
        hdu.verify('fix') 
        try: hdu.writeto('Galaxies/{0}/{1}.fits'.format(id_source, file_name), overwrite=True)
        except:
            subprocess.call('mkdir Galaxies/{0}'.format(id_source), shell = True)
            hdu.writeto('Galaxies/{0}/{1}.fits'.format(id_source, file_name), overwrite=True)
    return

In [5]:
galaxies_df = pd.DataFrame()
galaxies_df['source_id'] = ['test_new']
galaxies_df['RAJ2000'] = [150.0675914]
galaxies_df['DEJ2000'] = [2.2429768]

cutout_size = 10*u.arcsec
for _, row in tqdm(df_bundle.iterrows()):
    map_bundle = FitsUtils(row.Path, row.Filter, row.kind)
    generate_cutouts(map_bundle, cutout_size, galaxies_df)

1it [00:18, 18.57s/it]

Cutouts for filter HST f435w sci
No source in this cutout


2it [00:35, 17.34s/it]

Cutouts for filter HST f475w sci


3it [00:57, 19.50s/it]

...

Cutouts for filter JWST f444w sci


13it [04:10, 18.73s/it]

Cutouts for filter JWST f770w sci


14it [04:27, 19.09s/it]

Cutouts for filter JWST f1800w sci
No source in this cutout


---